# Demo

Injection of first order logic rules into a neural network for iris classification task.

Some imports.

In [1]:
from antlr4 import InputStream, CommonTokenStream
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.python.framework.random_seed import set_random_seed
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from psyki.datalog import Fuzzifier
from psyki.ski import ConstrainingInjector
from resources.dist.resources.DatalogLexer import DatalogLexer
from resources.dist.resources.DatalogParser import DatalogParser
from test import get_mlp
from test.resources import get_list_rules

Loading iris dataset and separation into train and test set.

In [2]:
x, y = load_iris(return_X_y=True, as_frame=True)
encoder = OneHotEncoder(sparse=False)
encoder.fit_transform([y])
dataset = x.join(y)

train, test = train_test_split(dataset, test_size=0.5, random_state=0)
train

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
3,4.6,3.1,1.5,0.2,0
149,5.9,3.0,5.1,1.8,2
98,5.1,2.5,3.0,1.1,1
6,4.6,3.4,1.4,0.3,0
68,6.2,2.2,4.5,1.5,1
...,...,...,...,...,...
9,4.9,3.1,1.5,0.1,0
103,6.3,2.9,5.6,1.8,2
67,5.8,2.7,4.1,1.0,1
117,7.7,3.8,6.7,2.2,2


Import FOL rules:

- PL <= 2.28 <- X = setosa
- PL > 2.28 ^ PW > 1.64 <- X = virginica
- PL > 2.28 ^ PW <= 1.64 <- X = versicolor

In [3]:
features_mapping = {
    'SL': 0,
    'SW': 1,
    'PL': 2,
    'PW': 3,
}
class_mapping = {
    'setosa': 0,
    'virginica': 1,
    'versicolor': 2
}

fuzzifier = Fuzzifier(class_mapping, features_mapping)
iris_rules = get_list_rules('iris')
formulae = {rule: DatalogParser(CommonTokenStream(DatalogLexer(InputStream(rule)))) for rule in iris_rules}

Injection of fuzzy logic function derived from FOL rules into a neural network.

In [4]:
set_random_seed(0)
input_features = Input((4,), name='Input')
network = get_mlp(input=input_features, output=3, layers=3, neurons=32, activation_function='relu',
                  last_activation_function='softmax')
model = Model(input_features, network)
injector = ConstrainingInjector(model, class_mapping, features_mapping)
injector.inject(formulae)
injector.predictor.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
injector.predictor.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input (InputLayer)              [(None, 4)]          0                                            
__________________________________________________________________________________________________
L_1 (Dense)                     (None, 32)           160         Input[0][0]                      
__________________________________________________________________________________________________
L_2 (Dense)                     (None, 32)           1056        L_1[0][0]                        
__________________________________________________________________________________________________
L_3 (Dense)                     (None, 3)            99          L_2[0][0]                        
____________________________________________________________________________________________

Training.

In [5]:
train_x, train_y = train.iloc[:,:-1], train.iloc[:,-1]
injector.predictor.fit(train_x, train_y, verbose=1, batch_size=4, epochs=30)

Epoch 1/30


2022-02-28 09:29:36.707510: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-02-28 09:29:36.707659: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


19/19 [==============================] - 0s 654us/step - loss: 1.0760 - accuracy: 0.5333
Epoch 2/30
19/19 [==============================] - 0s 673us/step - loss: 0.8121 - accuracy: 0.5867
Epoch 3/30
19/19 [==============================] - 0s 530us/step - loss: 0.7280 - accuracy: 0.5867
Epoch 4/30
19/19 [==============================] - 0s 577us/step - loss: 0.6740 - accuracy: 0.6133
Epoch 5/30
19/19 [==============================] - 0s 606us/step - loss: 0.6266 - accuracy: 0.5867
Epoch 6/30
19/19 [==============================] - 0s 550us/step - loss: 0.5803 - accuracy: 0.6267
Epoch 7/30
19/19 [==============================] - 0s 550us/step - loss: 0.5524 - accuracy: 0.6400
Epoch 8/30
19/19 [==============================] - 0s 604us/step - loss: 0.5276 - accuracy: 0.6400
Epoch 9/30
19/19 [==============================] - 0s 548us/step - loss: 0.5126 - accuracy: 0.6267
Epoch 10/30
19/19 [==============================] - 0s 573us/step - loss: 0.4988 - accuracy: 0.6267
Epoch 11/3

Removing the injected rules from the network.

In [6]:
new_network = Model(injector.predictor.input, injector.predictor.layers[-3].output)
new_network.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
new_network.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 4)]               0         
_________________________________________________________________
L_1 (Dense)                  (None, 32)                160       
_________________________________________________________________
L_2 (Dense)                  (None, 32)                1056      
_________________________________________________________________
L_3 (Dense)                  (None, 3)                 99        
Total params: 1,315
Trainable params: 1,315
Non-trainable params: 0
_________________________________________________________________


Evaluation.

In [7]:
test_x, test_y = test.iloc[:,:-1], test.iloc[:,-1]
new_network.evaluate(test_x, test_y)

3/3 [==============================] - 0s 636us/step - loss: 0.1601 - accuracy: 0.9867


[0.16007451713085175, 0.9866666793823242]

### Demo ends here
If you are reading it from `https://anonymous.4open.science/` there is a chance that the demo is duplicated.
Just ignore the following text.